# Aperture Size Tradebook

In [6]:
# stdlib
import math
from pathlib import Path

# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from architect import luts
from architect.libs import utillib
from architect.systems.optical import foreoptics, masks, sensors, spectrometers

## Setup
Run this section onwards when a parameter is updated.

### Parameters

In [7]:
# constants
target_wavelength = 1300 * unit.nm
orbital_altitude = 505 * unit.km
skew_angle = np.array([0, 0]) * unit.deg
slit_size = np.array([0.03, 38.]) * unit.mm         # width, height
focal_length = 50 * unit.mm

# variables
aperture_size = np.arange(start=1, stop=100, step=1) * unit.mm

### System Instantiation

In [8]:
# components
sensor = sensors.TauSWIR()
foreoptic = foreoptics.Foreoptic(
    focal_length=focal_length, diameter=aperture_size, image_diameter=10 * unit.mm
)
slit = masks.RectSlit(size=slit_size)

# systems
payload = spectrometers.HyperspectralImager(
    sensor=sensor, foreoptic=foreoptic, slit=slit
)

## Pipeline
The computational graph

In [9]:
radiance = luts.load("atmosphere/radiance_min")
snr = payload.get_signal_to_noise(radiance=radiance, wavelength=target_wavelength)
display(snr)

sensor_spatial_resolution = payload.get_spatial_resolution(
    wavelength=target_wavelength,
    target_distance=orbital_altitude,
    skew_angle=skew_angle[0],
).to(unit.m)
display(sensor_spatial_resolution)

#swath = payload.get_swath(altitude=orbital_altitude, skew_angle=skew_angle)
#display(swath)

<Quantity [1.21904034e-02, 4.87615756e-02, 1.09713402e-01, 1.95045691e-01,
           3.04758176e-01, 4.38850513e-01, 5.97322282e-01, 7.80172986e-01,
           9.87402053e-01, 1.21900883e+00, 1.47499260e+00, 1.75535255e+00,
           2.06008781e+00, 2.38919743e+00, 2.74268036e+00, 3.12053551e+00,
           3.52276169e+00, 3.94935764e+00, 4.40032203e+00, 4.87565344e+00,
           5.37535039e+00, 5.89941130e+00, 6.44783454e+00, 7.02061839e+00,
           7.61776107e+00, 8.23926069e+00, 8.88511531e+00, 9.55532291e+00,
           1.02498814e+01, 1.09687886e+01, 1.17120423e+01, 1.24796401e+01,
           1.32715796e+01, 1.40878584e+01, 1.49284738e+01, 1.57934234e+01,
           1.66827043e+01, 1.75963138e+01, 1.85342491e+01, 1.94965071e+01,
           2.04830849e+01, 2.14939794e+01, 2.25291875e+01, 2.35887059e+01,
           2.46725313e+01, 2.57806603e+01, 2.69130894e+01, 2.80698152e+01,
           2.92508339e+01, 3.04561420e+01, 3.16857356e+01, 3.29396109e+01,
           3.42177640e+01

<Quantity [800.93      , 400.465     , 266.97666667, 200.2325    ,
           160.186     , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5       ,
           151.5       , 151.5       , 151.5       , 151.5    

## Plots

In [10]:
aperture_size_label = f"Aperture Size [{aperture_size.unit}]"
snr_label = f"SNR"
sensor_spatial_resolution_label = f"Spatial Resolution [{sensor_spatial_resolution.unit}]"
#swath_label_x = f"Swath (across-track) [{swath.unit}]"
#swath_label_y = f"Swath (along-track) [{swath.unit}]"

data = {
    aperture_size_label: aperture_size,
    snr_label: snr,
    sensor_spatial_resolution_label: sensor_spatial_resolution,
    #swath_label_x: swath[:, 0],
    #swath_label_y: swath[:, 1],
}

df = pd.DataFrame.from_dict(data=data)
fig = plot.line(
    df=df,
    x=aperture_size_label,
    y=[  
        snr_label,
        sensor_spatial_resolution_label,
        #swath_label_x,
        #swath_label_y,
    ],
    title=None,
    dark=True,
)

fig.show()
display(df)
display(payload)

,Aperture Size [mm],SNR,Spatial Resolution [m]
0,1.0,0.012190,800.930000
1,2.0,0.048762,400.465000
2,3.0,0.109713,266.976667
3,4.0,0.195046,200.232500
4,5.0,0.304758,160.186000
...,...,...,...
94,95.0,109.760071,151.500000
95,96.0,112.077408,151.500000
96,97.0,114.418777,151.500000
97,98.0,116.784171,151.500000


,Value,Units
foreoptic,Foreoptic,None
slit,RectSlit,None
diffractor,None,None
sensor,TauSWIR,None
systems,list [4],None
dimensions,None,None
mass,None,None
volume,None,None
density,None,None
index,None,None
